In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_json('bright_data_1.json')

In [4]:
df.columns

Index(['input', 'error', 'price', 'section', 'row', 'ticket_type', 'Seat',
       'amount', 'availability'],
      dtype='object')

In [5]:
df

,input,error,price,section,row,ticket_type,Seat,amount,availability
0,{'url': 'https://www.ticketmaster.com/the-week...,This event has started or has already happened...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,{'url': 'https://concerts.livenation.com/alec-...,NaN,$35.00,GA3,N/A,Standard Ticket,N/A,1701.0,NaN
2,{'url': 'https://concerts.livenation.com/alec-...,NaN,$20.00,GA3,N/A,Summer's Live 4 Pack,N/A,8.0,NaN
3,{'url': 'https://concerts.livenation.com/alec-...,NaN,$65.00,GA3,N/A,GA Ticket + 40 Below Club Access,N/A,217.0,NaN
4,{'url': 'https://concerts.livenation.com/event...,NaN,$204.00,201,A,resale,2,1.0,1.0
...,...,...,...,...,...,...,...,...,...
816,{'url': 'https://concerts.livenation.com/event...,NaN,$107.00,GA4,N/A,resale,N/A,2.0,NaN
817,{'url': 'https://concerts.livenation.com/event...,NaN,$125.00,GA4,N/A,resale,N/A,1.0,NaN
818,{'url': 'https://concerts.livenation.com/event...,NaN,$90.00,GA4,N/A,resale,N/A,2.0,NaN
819,{'url': 'https://www.ticketmaster.com/pinstrip...,NaN,$15.00,GA03,N/A,Standard Ticket,N/A,1987.0,NaN


##### Transform Event Object

In [11]:
df['event_url'] = df['input'].map(lambda x: x['url'])

In [23]:
def strip_id(url):
    s = url[::-1]
    ind = s.find('/')
    if ind == -1: return None
    return s[:ind][::-1]

# url = "https://blah.com/skhd-sjbd-sdsd/event/7a439f374e9334"
# strip_id(url)

In [24]:
df['event_id'] = df['event_url'].map(lambda x: strip_id(x))

##### Transform Price

In [32]:
x = "$35.00"

In [33]:
x.replace("$","")

'35.00'

In [34]:
def price_t(s):
    if s == "NaN":
        return pd.NA
    try:
        return float(s.replace("$", ""))
    except:
        return pd.NA

In [35]:
df['price_s'] = df['price']

In [36]:
df['price'] = df['price_s'].map(lambda x: price_t(x))

In [38]:
x = df['price']

In [42]:
'total price=NA: ', sum(pd.isna(df['price']))

('total price=NA: ', 664)

In [43]:
df.shape

(821, 12)

### Aggregate

In [25]:
df.groupby(['event_id', 'section']).agg(['count','sum'])['amount']

count     sum
event_id         section               
0E005C56E48E2B02 201          3     3.0
                 202          1     1.0
                 203          2     2.0
                 204          5     5.0
                 205          1     1.0
                 206          2     2.0
                 302          3     3.0
                 303          1     1.0
                 304          3     3.0
                 305          1     1.0
                 306          3     3.0
                 401         58    58.0
                 402         66    66.0
                 GA4          4    18.0
1D005B289CA82895 GA03         1  1987.0
3A005CA3ECA77FB3 GA3          3  1926.0

In [45]:
df.groupby(['event_id', 'section']).agg(['mean','min','max'])['price'].round(2)

mean    min     max
event_id         section                       
0E005C56E48E2B02 201      151.33  125.0  204.00
                 202       70.00   70.0   70.00
                 203       55.50   55.5   55.50
                 204       55.50   55.5   55.50
                 205       70.00   70.0   70.00
                 206      185.00  185.0  185.00
                 302      101.54   70.0  117.31
                 303       55.50   55.5   55.50
                 304       60.33   55.5   70.00
                 305       70.00   70.0   70.00
                 306       58.33   55.5   64.00
                 401       55.50   55.5   55.50
                 402       55.50   55.5   55.50
                 GA4       92.38   47.5  125.00
1D005B289CA82895 GA03      15.00   15.0   15.00
3A005CA3ECA77FB3 GA3       40.00   20.0   65.00